In [7]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")



In [8]:
model=ChatGroq(
    model_name="llama-3.3-70b-versatile",
    api_key=groq_api_key
)
response=model.invoke("Hello, how are you?")
print(response)

content="Hello. I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 41, 'total_tokens': 89, 'completion_time': 0.205995378, 'prompt_time': 0.002162741, 'queue_time': 0.058274876999999996, 'total_time': 0.208158119}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'finish_reason': 'stop', 'logprobs': None} id='run--e1337914-6f16-4e14-ad1b-27bafc65a0f9-0' usage_metadata={'input_tokens': 41, 'output_tokens': 48, 'total_tokens': 89}


In [9]:
model.invoke([
    HumanMessage(content="What is the capital of France?")])

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 42, 'total_tokens': 50, 'completion_time': 0.029090909, 'prompt_time': 0.008738453, 'queue_time': 0.052715725, 'total_time': 0.037829362}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'finish_reason': 'stop', 'logprobs': None}, id='run--31d88152-452e-4d6a-8f73-a9285d562499-0', usage_metadata={'input_tokens': 42, 'output_tokens': 8, 'total_tokens': 50})

In [10]:
from langchain_core.messages import AIMessage
response = model.invoke([
    HumanMessage(content="What is the capital of France?")
])
print(response)

content='The capital of France is Paris.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 42, 'total_tokens': 50, 'completion_time': 0.029090909, 'prompt_time': 0.001823128, 'queue_time': 0.056122202, 'total_time': 0.030914037}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'finish_reason': 'stop', 'logprobs': None} id='run--60c35be0-8667-4bd3-b79d-1e74ed9fdfb0-0' usage_metadata={'input_tokens': 42, 'output_tokens': 8, 'total_tokens': 50}


In [18]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_history = RunnableWithMessageHistory(model, get_session_history)

In [19]:
config={"configurable": {"session_id":"session1"}}
with_history.invoke([
    HumanMessage(content="What is the capital of France?")],config=config
)

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 42, 'total_tokens': 50, 'completion_time': 0.029090909, 'prompt_time': 0.002066022, 'queue_time': 0.051596238, 'total_time': 0.031156931}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'finish_reason': 'stop', 'logprobs': None}, id='run--7a2973e9-453b-444a-a8cf-20df30c4ace7-0', usage_metadata={'input_tokens': 42, 'output_tokens': 8, 'total_tokens': 50})

In [29]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="input", role="human")
    ]
)


chain = prompt | model


response = chain.invoke({"input":[HumanMessage(content="Tell me about artificial intelligence in 3 sentences.")]})

print(response)

content='Artificial intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. AI systems use algorithms and data to enable machines to think and act like humans, and they have numerous applications in areas like healthcare, finance, and transportation. From virtual assistants like Siri and Alexa to self-driving cars and personalized product recommendations, AI is increasingly being integrated into our daily lives, transforming the way we live, work, and interact with technology.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 51, 'total_tokens': 156, 'completion_time': 0.381818182, 'prompt_time': 0.002328125, 'queue_time': 0.052426985, 'total_time': 0.384146307}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'finish_reason': 'stop', 'logprobs': None} id='run--523ab527-20b6-49

In [ ]:
from langchain_core.messages import AIMessage,SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=30,
    strategy="last",
    token_counter=model.get_num_tokens,include_system=True,allow_partial=False,start_on="human"
)


In [ ]:
from operator  import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain = (RunnablePassthrough.assign(messaages=itemgetter("messages")) | trimmer | model)
chain.invoke("messages")

Vector Store and Retriever

In [61]:
from langchain_core.documents import Document
documents = [
    Document(page_content="Paris is the capital of France.", metadata={"source": "wiki"}),
    Document(page_content="Berlin is the capital of Germany.", metadata={"source": "wiki"}),
    Document(page_content="Tokyo is the capital of Japan.", metadata={"source": "wiki"})
]

In [63]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
load_dotenv()
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore=Chroma.from_documents(documents,embedding=embeddings)

In [66]:
vectorstore.similarity_search("paris is capital of?",k=1)

[Document(id='1be538d0-c72c-4993-8467-a333e5ce070a', metadata={'source': 'wiki'}, page_content='Paris is the capital of France.')]

In [70]:
from typing import List
from langchain_core.runnables import RunnableLambda
retriever=RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["paris", "berlin", "tokyo"])

[[Document(id='1be538d0-c72c-4993-8467-a333e5ce070a', metadata={'source': 'wiki'}, page_content='Paris is the capital of France.')],
 [Document(id='75e46d8f-9f91-4998-977a-3cd23fce5d3b', metadata={'source': 'wiki'}, page_content='Berlin is the capital of Germany.')],
 [Document(id='6087db16-f28f-4b5d-aebf-346f7e6d1341', metadata={'source': 'wiki'}, page_content='Tokyo is the capital of Japan.')]]